# Importando as bibliotecas

In [17]:
from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import confusion_matrix
import pandas as pd
import numpy as np
from collections import Counter
import os
import sys
sys.path.insert(0, os.path.abspath('../Util'))
from dados import ProcessarDados

# Dataset

In [18]:
procData = ProcessarDados("../dataset/norm_bin_10_FEATURES_M17_CM6b_TH199.csv")

# Funções

In [19]:
# funcção que treina e testa o modelo armazenando as métricas
# retorna um dicionário cotendo os valores das métricas de cada rodada
def treinamento_teste(epocas = 10, k_folds = 5, exibir_matriz_confusao=False, exibir_metricas=False):
    
    #array para armazenar as das métricas de cada rodada
    resultados_accuracy = []
    resultados_precision = []
    resultados_recall = []
    resultados_f1 = []
    resultados_parametros = []
    
    #dicionário das métricas
    resultados_gerais = {}

    for i in range(epocas):
        # divisão os dados 
        X_train, X_test, y_train, y_test = procData.holdout(0.2)
        #print(Counter(y_test))

        # realizando o grid search para encontrar a melhor combinação entre o C, gamma e Kernel, 
        # considerando a acurácia (taxa de acerto)
        # aqui o método GridSearchCV é configurado para subdividir os dados de treino em k_folds
        clf = svm.SVC(random_state = None)
        grid_svm = GridSearchCV(clf, param_grid, cv=k_folds, scoring='accuracy', verbose=0)
        grid_svm.fit(X_train, y_train)

        # Treinando do modelo com os melhores parametros encontrados
        C_best = grid_svm.best_estimator_.C
        gamma_best = grid_svm.best_estimator_.gamma
        kernel_best = grid_svm.best_estimator_.kernel

        SVM = svm.SVC(random_state = None, C = C_best, gamma = gamma_best, kernel = kernel_best)
        SVM.fit(X_train, y_train)

        #testando o modelo
        y_pred = SVM.predict(X_test)
        if exibir_matriz_confusao:
            print(confusion_matrix(y_test, y_pred))

        # calculado as metricas
        accuracy = metrics.accuracy_score(y_test, y_pred)
        precision = metrics.precision_score(y_test, y_pred)
        recall = metrics.recall_score(y_test, y_pred)
        f1_score = 2 * (precision * recall) / (precision + recall)

        # armazenando as métricas
        resultados_accuracy.append(accuracy)
        resultados_precision.append(precision)
        resultados_recall.append(recall)
        resultados_f1.append(f1_score)

        best_parametros = "C: "+ str(C_best)+ " | Gamma: "+ str(gamma_best)+ " | Kernel: "+ str(kernel_best);
        resultados_parametros.append(best_parametros)


        if exibir_metricas:
            print("Rodada: #",i)
            print(best_parametros)
            print("Accuracy:",accuracy)
            print("Precision:",precision)
            print("Recall:",recall)
            print("F1-Score:",f1_score)
            print("\n")

            
    resultados_gerais['accuracy'] = resultados_accuracy
    resultados_gerais['precision'] = resultados_precision
    resultados_gerais['recall'] = resultados_recall
    resultados_gerais['f1'] = resultados_f1
    
    return resultados_gerais

In [35]:
def tabelaMetricas(nome_modelo, dict_metricas, rodadas=False, salvarResultados=True):
        
    print ("============================================== "+nome_modelo+" =================================================")
    print ("=================================== TABELA DE MÉTRICAS DO MODELO ===================================")
    
    if(rodadas==False):
        print ("\t Accuracy \t|\t Precision \t|\t Recall \t|\t F1-Score")
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['accuracy'], axis=0), np.std(dict_metricas['accuracy'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['precision'], axis=0), np.std(dict_metricas['precision'], axis=0)),end='    ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['recall'], axis=0), np.std(dict_metricas['recall'], axis=0)),end='       ')
        print ("      %.2f +- %.2f" % (np.mean(dict_metricas['f1'], axis=0), np.std(dict_metricas['f1'], axis=0)))
        print ("====================================================================================================")
        
    if(salvarResultados):
        # save to npy file
        np.save('../resultados/resultados_'+nome_modelo+'.npy', dict_metricas)

# Definindo os parâmetros

In [38]:
c_range = [10, 20, 30, 40, 50]
gammas = [0.01, 0.02, 0.03, 0.04, 0.05]
kernels = ["rbf"]
param_grid = {'C': c_range, 'gamma': gammas, 'kernel': kernels}

epocas = 50
k_folds = 5
exibir_matriz_confusao = True
exibir_metricas = True
salvarResultados = True
rodadas=False

# Treinando e obtendo as métricas do modelo

In [39]:
# treinando o modelo
dict_metricas = treinamento_teste(epocas, k_folds, exibir_matriz_confusao, exibir_metricas)
tabelaMetricas('SVM',dict_metricas, rodadas, salvarResultados)

[[19  7]
 [ 8 86]]
Rodada: # 0
C: 30 | Gamma: 0.02 | Kernel: rbf
Accuracy: 0.875
Precision: 0.9247311827956989
Recall: 0.9148936170212766
F1-Score: 0.9197860962566845


[[27  3]
 [ 5 85]]
Rodada: # 1
C: 10 | Gamma: 0.02 | Kernel: rbf
Accuracy: 0.9333333333333333
Precision: 0.9659090909090909
Recall: 0.9444444444444444
F1-Score: 0.9550561797752809


[[25 11]
 [ 0 84]]
Rodada: # 2
C: 10 | Gamma: 0.04 | Kernel: rbf
Accuracy: 0.9083333333333333
Precision: 0.8842105263157894
Recall: 1.0
F1-Score: 0.9385474860335195


[[18  9]
 [ 4 89]]
Rodada: # 3
C: 30 | Gamma: 0.03 | Kernel: rbf
Accuracy: 0.8916666666666667
Precision: 0.9081632653061225
Recall: 0.956989247311828
F1-Score: 0.9319371727748691


[[22  9]
 [ 6 83]]
Rodada: # 4
C: 40 | Gamma: 0.02 | Kernel: rbf
Accuracy: 0.875
Precision: 0.9021739130434783
Recall: 0.9325842696629213
F1-Score: 0.9171270718232043


[[25  5]
 [ 2 88]]
Rodada: # 5
C: 30 | Gamma: 0.01 | Kernel: rbf
Accuracy: 0.9416666666666667
Precision: 0.946236559139785
Recall: 0

[[21 13]
 [ 4 82]]
Rodada: # 47
C: 50 | Gamma: 0.04 | Kernel: rbf
Accuracy: 0.8583333333333333
Precision: 0.8631578947368421
Recall: 0.9534883720930233
F1-Score: 0.9060773480662985


[[26  9]
 [ 5 80]]
Rodada: # 48
C: 30 | Gamma: 0.05 | Kernel: rbf
Accuracy: 0.8833333333333333
Precision: 0.898876404494382
Recall: 0.9411764705882353
F1-Score: 0.9195402298850575


[[19  8]
 [ 4 89]]
Rodada: # 49
C: 10 | Gamma: 0.03 | Kernel: rbf
Accuracy: 0.9
Precision: 0.9175257731958762
Recall: 0.956989247311828
F1-Score: 0.9368421052631579


============================================== SVM =================================================
=================================== TABELA DE MÉTRICAS DO MODELO ===================================
	 Accuracy 	|	 Precision 	|	 Recall 	|	 F1-Score
      0.90 +- 0.03             0.91 +- 0.03          0.96 +- 0.02             0.93 +- 0.02
